Hola **Jesus**!

Soy **Patricio Requena** 👋. Es un placer ser el revisor de tu proyecto el día de hoy!

Revisaré tu proyecto detenidamente con el objetivo de ayudarte a mejorar y perfeccionar tus habilidades. Durante mi revisión, identificaré áreas donde puedas hacer mejoras en tu código, señalando específicamente qué y cómo podrías ajustar para optimizar el rendimiento y la claridad de tu proyecto. Además, es importante para mí destacar los aspectos que has manejado excepcionalmente bien. Reconocer tus fortalezas te ayudará a entender qué técnicas y métodos están funcionando a tu favor y cómo puedes aplicarlos en futuras tareas. 

_**Recuerda que al final de este notebook encontrarás un comentario general de mi parte**_, empecemos!

Encontrarás mis comentarios dentro de cajas verdes, amarillas o rojas, ⚠️ **por favor, no muevas, modifiques o borres mis comentarios** ⚠️:


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si todo está perfecto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>

Puedes responderme de esta forma:
<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class=“tocSkip”></a>
</div>

----

## Descripción del proyecto

El presente proyecto tiene el objetivo de identificar el 'churn' ó la tasa de cancelación de los usuarios de la operadora de telecomunicaciones Interconnect. Para ello, se requiere diseñar un prototipo de un modelo de aprendizaje automático que pueda predecir que clientes van a cancelar con base a la informacion recopilada.

## Parte 1. Inicialización

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
import optuna
import logging

In [2]:
logging.basicConfig(level=logging.WARNING)
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [3]:
#contract = pd.read_csv('C:/Users/jesus/OneDrive/Documentos/Programacion/Triple Ten/Sprint 17/Proyecto/Data/contract.csv', parse_dates = ['BeginDate'])
contract = pd.read_csv('/datasets/final_provider/contract.cs', parse_dates = ['BeginDate'])
#personal = pd.read_csv('C:/Users/jesus/OneDrive/Documentos/Programacion/Triple Ten/Sprint 17/Proyecto/Data/personal.csv')
personal = pd.read_csv('/datasets/final_provider/personal.csv')
#internet = pd.read_csv('C:/Users/jesus/OneDrive/Documentos/Programacion/Triple Ten/Sprint 17/Proyecto/Data/internet.csv')
internet = pd.read_csv('/datasets/final_provider/internet.csv')
#phone = pd.read_csv('C:/Users/jesus/OneDrive/Documentos/Programacion/Triple Ten/Sprint 17/Proyecto/Data/phone.csv')
phone = pd.read_csv('/datasets/final_provider/phone.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/final_provider/contract.cs'

## Parte 2. EDA

In [ ]:
contract.head(5)

In [ ]:
contract['PaymentMethod'].unique()

In [ ]:
contract['EndDate'] = pd.to_datetime(contract['EndDate'], errors = 'coerce')

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteración)</b> <a class=“tocSkip”></a>

Buen trabajo convirtiendo el tipo de variable de la columna *EndDate*, esto es clave para tener un flujo de trabajo correcto.

</div>

In [ ]:
contract['EndDate'].isna().sum()

In [ ]:
contract.info()

In [ ]:
contract.describe()

In [ ]:
contract.head()

In [ ]:
personal.head(5)

In [ ]:
personal.info()

In [ ]:
personal.describe()

In [ ]:
internet.head(5)

In [ ]:
internet['InternetService'].unique()

In [ ]:
internet.info()

In [ ]:
internet.describe()

In [ ]:
phone.head(5)

In [ ]:
phone.info()

In [ ]:
phone.describe()

In [ ]:
dfs = [contract, personal, internet, phone]
df = reduce(lambda left, right: pd.merge(left, right, on = 'customerID', how = 'inner'), dfs)
df

In [ ]:
df['customerID'].duplicated().sum()

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors = 'coerce')

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteración)</b> <a class=“tocSkip”></a>

Buen trabajo Jesús, es clave la conversión de los datos de la columna *TotalCharges*, es muy común encontrar información con tipos de datos que no correspondan, pero con un adecuado EDA se pueden identificar y hacer los ajustes correctos.

</div>

In [ ]:
df['MonthlyCharges'].describe()

In [ ]:
df['TotalCharges'].describe()

In [ ]:
df = df.dropna(subset = ['TotalCharges'])

In [ ]:
df.info()

In [ ]:
df.drop(columns='customerID')

In [ ]:
sns.kdeplot(df['TotalCharges'], fill=True, color='blue')

plt.title('Gráfico de densidad de TotalCharges')
plt.xlabel('TotalChargues')
plt.ylabel('Densidad')

plt.show()

In [ ]:
sns.kdeplot(df['MonthlyCharges'], fill=True, color='blue')

plt.title('Gráfico de densidad de MonthlyCharges')
plt.xlabel('MonthlyCharges')
plt.ylabel('Densidad')

plt.show()

In [ ]:
df = df.drop(columns=['customerID', 'SeniorCitizen'])

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteración)</b> <a class=“tocSkip”></a>

Excelente idea incluir gráficas para identificar patrones iniciales de los datos. 

</div>

### Analisis Paso 2

- Los tipos de valores para las columnas EndDate y TotalChargues eran incorrectas. EndDate se cambió a datetime y TotalChargues a int. 
- Se eliminó la columna 'CustomerID' dado que lejos de ayudar, perjudica que esta serie permanezca en la data, esto se debe a que el modelo busca encontrar patrones y esta columna lo que tiene son valores unicos.
- Hay un claro sezgo a la izquierda en los datos de 'TotalChargues'.
- En 'MonthlyChargues' parece observarse una distribución normal.

## Paso 3. Normalización y codificación

In [ ]:
df.columns

In [ ]:
cat_features = ['Type', 'PaperlessBilling', 'PaymentMethod', 'gender', 'Partner', 'Dependents',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']

df = pd.get_dummies(df, columns = cat_features, drop_first = True)

In [ ]:
numeric_features = ['MonthlyCharges', 'TotalCharges']
scaler = StandardScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])

### Analisis Paso 3
- La data que requeria codificacion era la categorica, en las columnas ['Type', 'PaperlessBilling', 'PaymentMethod', 'gender', 'Partner', 'Dependents', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines'] 
- Se usó el metodo OHE (One Hot Encoder) con el metodo 'get_dummies' con el parametro 'drop_first' en 'True' con el fin de evitar inconvenientes en le procesamiento de la información al dejar columnas innecesarias. 
- Para la estandarización de los valores numericos se usó el metodo StandardScaler. Estos valores se encuentran en las columnas 'TotalChargues' y 'MonthlyChargues'-

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Muy bien con el encoding, los modelos necesitan de datos numéricos para poder ser entrenados por lo que este paso es muy importante
</div>

## Paso 4. Continuación EDA

In [ ]:
df['churn'] = df['EndDate'].notnull().astype(int)
df

In [ ]:
#reference_date = df['BeginDate'].max()
#df['duracion_dias'] = (df['reference_date'] - df['BeginDate']).dt.days
#df

In [ ]:
df.dtypes

In [ ]:
def make_features(df, column):
    df[f'year {column}'] = df[column].dt.year 
    df[f'month {column}'] = df[column].dt.month
    df[f'day {column}'] = df[column].dt.day
    df[f'dayofweek {column}'] = df[column].dt.dayofweek

make_features(df, 'BeginDate')
make_features(df, 'EndDate')

In [ ]:
print(df['year BeginDate'].min())
print(df['year BeginDate'].max())
print(df['year EndDate'].min())
print(df['year EndDate'].max())

In [ ]:
df.columns

In [ ]:
correlation_with_churn = df.corr()['churn'].sort_values(ascending=False)

plt.figure(figsize=(10, 12)) 
sns.heatmap(correlation_with_churn.to_frame(), annot=True, cmap='rocket', cbar=True, linewidths=0.5)

plt.title('Correlations with Churn', fontsize=16)
plt.xlabel('Correlation', fontsize=12)
plt.ylabel('Variables', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
columns = ['year EndDate', 'month EndDate', 'day EndDate', 'dayofweek EndDate']

fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharey=True)

for i, col in enumerate(columns):
    sns.kdeplot(df[col].dropna(), fill=True, color='blue', alpha=0.6, ax=axes[i])
    axes[i].set_title(col, fontsize=12)
    axes[i].set_xlabel('Valores', fontsize=10)
    axes[i].set_ylabel('Densidad', fontsize=10 if i == 0 else 0)

plt.tight_layout()
plt.suptitle('Distribuciones de Densidad de las Columnas', fontsize=16, y=1.05)
plt.show()

In [ ]:
columns = ['year BeginDate', 'month BeginDate', 'day BeginDate', 'dayofweek BeginDate']

fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharey=True)

for i, col in enumerate(columns):
    sns.kdeplot(df[col].dropna(), fill=True, color='blue', alpha=0.6, ax=axes[i])
    axes[i].set_title(col, fontsize=12)
    axes[i].set_xlabel('Valores', fontsize=10)
    axes[i].set_ylabel('Densidad', fontsize=10 if i == 0 else 0)

plt.tight_layout()
plt.suptitle('Distribuciones de Densidad de las Columnas', fontsize=16, y=1.05)
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))  # Tamaño ajustado para dos columnas
sns.heatmap(
    df[['BeginDate', 'EndDate']].isnull(),  # Filtrar columnas
    cbar=True,
    cbar_kws={'label': 'Missing Data'},
    cmap='viridis'
)
plt.title("Missing Data Heatmap (Churn & Duracion de Dias)", fontsize=16)
plt.show()

<div class="alert alert-block alert-danger">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Para esta parte de las gráficas debes dejar la redacción de un comentario con la interpretación de las mismas. Particularmente la última gráfica antes de este comentario no es claro en lo que aporta al proyecto, también te recomendaría revisar la correlación de todas las variables entre sí además de sólo con la variable objetivo. 
</div>

In [ ]:
df = df.drop(columns=['EndDate', 'BeginDate', 'year EndDate', 'month EndDate', 'day EndDate', 'dayofweek EndDate'], axis=1)

In [ ]:
df.head()

In [ ]:
correlation_with_churn = df.corr()['churn'].sort_values(ascending=False)

plt.figure(figsize=(10, 12)) 
sns.heatmap(correlation_with_churn.to_frame(), annot=True, cmap='rocket', cbar=True, linewidths=0.5)

plt.title('Correlations with Churn', fontsize=16)
plt.xlabel('Correlation', fontsize=12)
plt.ylabel('Variables', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
correlation_threshold = 0.1  
target_variable = 'churn'  

# Obtener las correlaciones con la variable objetivo
correlations = df.corr()[target_variable]

# Filtrar las columnas cuya correlación absoluta sea mayor o igual al umbral
columns_to_keep = correlations[correlations.abs() >= correlation_threshold].index

# Crear un nuevo DataFrame solo con las columnas seleccionadas
df = df[columns_to_keep]

print(f"Columnas seleccionadas (correlación >= {correlation_threshold} o <= -{correlation_threshold}):")
print(df.columns)

### Analisis Paso 4.
1. Se creo la columna 'churn' la cual contiene valores binarios que representan si el cliente sigue activo o no. 1 = No activo, 0 = Activo.
2. Las fechas son cruciales para el fin de este proyecto, predecir el churn, por lo que se dió un procesamiento a estas caracteristicas para generar nuevas, esto debido a que los valores datetime no pueden ser procesados y deben ser convertidos a valores numericos. Por lo que se obtuvieron la columnas 'year', 'month', 'day' y 'weekday' tanto de 'BeginDate' como de 'EndDate'.
3. Se generó un grafico de correlación multivariable, solo se dió atención a la correlación entre 'churn' y el resto de caractersticas. Los resultados más destacables fueron: 
    * Como se esperaba, varias caracteristicas de temporalidad estan altamente correlacionadas con churn.
    * Era una posibilidad introducir la variable ternure o duración de días a la data; sin embargo, se obto por no hacerlo debido a que esto iba a perjudicar más que a beneficias. Me explico, para hacer una aproximación de esta información sin usar EndDate, se tenía que usar una fecha de referencia, cualquiera esta hubiese sido (la mas recomendable era la más proxima ó la maxima registrada) el calculo de la duración habría sido sezgado debido a que todos los valores no nulos son valores que se conocen, y por tanto son exactos, por lo que rellenarlos con data diferente habría implicado deliberadamente sezgar nuestra información. Tal vez este enfoque habría sido util si la mayoría de los valores hubiesen sido nulos (usuarios registrados activos), sin embargo, este no es el caso, la mayoría de los usuarios registrados tienen valores no nulos, por lo que rellenar valos nulos con referencias arbitrarias habría representado un sezgo a los datos.
4. Podemos deividir en general es dos categorias a las caracteristicas, unas con correlación negativa y otras con positiva:
    * Sobre las positivas las mas relevantes son 3: ['year BeginDate', 'PaymenthMethod_Electronic check', 'InternetService_Fiber optic'] con valores de [.34, .27, .26] cada uno respectivamente. Esto implica que a mayores valores tengan estas caracteristicas, mayor será la probabilidad de cancelación del cliente.
    * Por el lado de las variables con relación negativa, las más destacadas fueron ['TotalChargues', 'Type_two year', 'OnlineSecurity_Yes', 'TechSupport_Yes', 'OnlineBackup_Yes'], con valores de ['.34', '.29', '.28', '.27', '.26'] respectivamente. Esta relación inversa indica que a mayores valores en estas variables, menor probabilidad habra de que se den de baja los usuarios.

5. Se eliminaron las variables que tuvieran un coeficiente de correlación menor a .1 debido a su irrelevancia al modelo y a evitar el riesgo de que estas producieran 'ruido' y distrajeran al modelo de las variables relevantes.
6. Se realizaron analisis de funciones de densidad para las variables temporales, tanto para las de inicio como del fin de vida de los usuarios:
    * Lo más relevante que observe es que hay temporadas bien diferenciadas del comportamiento de los usuarios para darse de baja. La temporada inicia a finales de septiembre y termina a mediados de febrero. Esto con tendencia a la baja según el muestreo disponible que consta de aproximadamente de poco menos de 2 años. Esto es consistente con la distribución de 'month EndDate' en la que como en la variable de año, aquí se tiene una distribución binomial al principio y al final de sus valores, los cuales representan los meses del mes con su respectivo número (1 al 12), lo que significa que los meses en que se dan de baja las personas son a principio y al final del año.
    * Respecto a las variables temporales de BeginDate, en 'year BeginDate' tenemos una distribución dimodal, el primer pico se genera entre los años 2013 y 2015 aproximadamente, y el segundo entre los años 2018 y principios del 2020.

<div class="alert alert-block alert-danger">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Cómo mencioné antes deberías revisar la correlación entre cada variable (Puedes omitir las que fueron codificadas si ves que esto te genera una matriz muy grande) y debes también revisar el balance de clases en la variable objetivo `churn` para que sepas si debes aplicar alguna técnica luego o ajustar los modelos según sea el caso
</div>

<div class="alert alert-block alert-success">
<b>Comentario del revisor (2da Iteracion)</b> <a class=“tocSkip”></a>

Correcto, siempre procura dejar la redacción con la interpretación de las gráficas que estás mostrando
</div>

## Paso 5. Segmentación de datos en 3 conjuntos

In [ ]:
x = df.drop(columns = 'churn', axis = 1)
y = df['churn']

x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size = 0.4, random_state = 12345)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size = 0.5, random_state = 12345)

x_test = x_test.iloc[1:]
y_test = y_test.iloc[1:]

print(f'Tamaño de caracteristicas en entrenamiento {x_train.shape}')
print(f'Tamaño de objetivo en entrenamiento {y_train.shape}')
print(f'Tamaño de caracteristicas en validación {x_val.shape}')
print(f'Tamaño de objetivo en validación {y_val.shape}')
print(f'Tamaño de caracteristicas en test {x_test.shape}')
print(f'Tamaño de objetivo en test {y_test.shape}')

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Muy bien, con esto tienes la división correcta para probar cada parte de la evaluación
</div>

## Paso 6. Análisis de balance de clases. Una o más estrategias. +

Utilizaré el metodo en el que indico en un parametro que quiero que las clases esten balanceadas. 

## Paso 7. Selecciona los modelos con los que vas a trabajar y vas comparar.

In [ ]:
models, predictions = LazyClassifier().fit(x_train, x_val, y_train, y_val)
print(models[:10])

Elegiré primero los modelos que mejor me han dado resultado y luego tomare 1 del metodo LazyClassifier:
- RandomForestClassifier
- DecisionTreeClassifier
- GaussianNB

### RandomForestClassifier

In [ ]:
# Asumiendo que tienes un DataFrame llamado x_train con las características
# y un vector y_train con las etiquetas (churn)
# Supongamos que 'y_train' es la variable objetivo (churn), que está en el rango [0, 1]

# Calculamos la matriz de correlación entre las características y el churn
correlation_matrix = x_train.corrwith(y_train)

# Definir un umbral para filtrar las características con mayor correlación (por ejemplo, > 0.1 en valor absoluto)
threshold = 0.1
important_features = correlation_matrix[correlation_matrix.abs() > threshold]

# Asignar pesos a las muestras
# En este caso, las muestras que dependen más de características con alta correlación reciben un mayor peso
sample_weights = np.zeros(len(x_train))

for feature in important_features.index:
    # Si la correlación de la característica con el churn es positiva o negativa,
    # las muestras con valores más altos de esa característica recibirán más peso.
    # Podemos considerar tanto las correlaciones positivas como negativas.
    sample_weights += np.abs(x_train[feature]) * np.abs(correlation_matrix[feature])

# Normalizar los pesos si es necesario (esto es opcional, dependiendo de tus necesidades)
sample_weights = sample_weights / sample_weights.sum()

# Ahora puedes usar `sample_weights` al entrenar tu modelo

<div class="alert alert-block alert-danger">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

¿Cuál es la razón de este código? ¿Por qué se generan estos `weights`? ¿En base a que defines el umbral o threshold?
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor (2da Iteracion)</b> <a class=“tocSkip”></a>

Sigues teniendo esta parte que no es claro el porque y que aporta al proyecto
</div>

In [ ]:
def objective_1(trial):
    # Definir los hiperparámetros a optimizar
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 20, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)
    class_weight = trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample', None])

    # Crear y entrenar el clasificador con los parámetros sugeridos
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        class_weight=class_weight,
        random_state=42
    )

    model.fit(x_train, y_train, sample_weights)

    # Predicciones y evaluación
    y_pred = model.predict(x_val)
    y_prob = model.predict_proba(x_val)[:, 1]  # Probabilidades para AUC-ROC

    # Calcular las métricas
    auc_roc = roc_auc_score(y_val, y_prob)
    accuracy = accuracy_score(y_val, y_pred)

    # Optuna optimiza para la métrica que se devuelve, en este caso, AUC-ROC
    return auc_roc

# Crear el estudio de Optuna
study_1 = optuna.create_study(direction='maximize')

# Ejecutar la optimización
study_1.optimize(objective_1, n_trials=100)

# Imprimir el mejor conjunto de hiperparámetros y las métricas correspondientes
print("Best hyperparameters:", study_1.best_params)
print("Best AUC-ROC:", study_1.best_value)

# Evaluar el modelo con los mejores hiperparámetros
best_params_1 = study_1.best_params
best_model_1 = RandomForestClassifier(**best_params_1, random_state=42)
best_model_1.fit(x_train, y_train)

# Predicciones y métricas finales
y_pred_1 = best_model_1.predict(x_val)
y_prob_1 = best_model_1.predict_proba(x_val)[:, 1]

final_accuracy_1 = accuracy_score(y_val, y_pred_1)
final_auc_roc_1 = roc_auc_score(y_val, y_prob_1)

print("Final Accuracy:", final_accuracy_1)
print("Final AUC-ROC:", final_auc_roc_1)

#### Prueba con test

In [ ]:
model_1 = RandomForestClassifier(n_estimators= 108, max_depth= 15, min_samples_split= 11, min_samples_leaf= 5, class_weight= 'balanced', random_state=12345)
model_1.fit(x_train, y_train, sample_weights)
pred_1 = model_1.predict(x_test)
prob_1 = best_model_1.predict_proba(x_test)[:, 1]

accuracy_1 = accuracy_score(y_test, pred_1)
auc_roc_1 = roc_auc_score(y_test, prob_1)

print(f'Valor de accuracy para RandomForestClassifier {accuracy_1}')
print(f'Valor de AUC_ROC para RandomForestClassifier {auc_roc_1}')

### DecisionTreeClassifier

In [ ]:
def objective_2(trial):
    # Definir los hiperparámetros a optimizar
    max_depth = trial.suggest_int('max_depth', 2, 25)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 25)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)

    # Crear y entrenar el clasificador con los parámetros sugeridos
    model = DecisionTreeClassifier(
        max_depth = max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        class_weight = 'balanced',
        random_state=42
    )

    model.fit(x_train, y_train)

    # Predicciones y evaluación
    y_pred = model.predict(x_val)
    y_prob = model.predict_proba(x_val)[:, 1]  # Probabilidades para AUC-ROC

    # Calcular las métricas
    auc_roc = roc_auc_score(y_val, y_prob)
    accuracy = accuracy_score(y_val, y_pred)

    # Optuna optimiza para la métrica que se devuelve, en este caso, AUC-ROC
    return auc_roc

# Crear el estudio de Optuna
study_2 = optuna.create_study(direction='maximize')

# Ejecutar la optimización
study_2.optimize(objective_2, n_trials=50)

# Imprimir el mejor conjunto de hiperparámetros y las métricas correspondientes
print("Best hyperparameters:", study_2.best_params)
print("Best AUC-ROC:", study_2.best_value)

# Evaluar el modelo con los mejores hiperparámetros
best_params_2 = study_2.best_params
best_model_2 = DecisionTreeClassifier(**best_params_2, random_state=42)
best_model_2.fit(x_train, y_train)

# Predicciones y métricas finales
y_pred_2 = best_model_2.predict(x_val)
y_prob_2 = best_model_2.predict_proba(x_val)[:, 1]

final_accuracy_2 = accuracy_score(y_val, y_pred_2)
final_auc_roc_2 = roc_auc_score(y_val, y_prob_2)

print("Final Accuracy:", final_accuracy_2)
print("Final AUC-ROC:", final_auc_roc_2)

#### Prueba con conjunto test

In [ ]:
model_2 = DecisionTreeClassifier(max_depth= 6, min_samples_split= 18, min_samples_leaf= 2)
model_2.fit(x_train, y_train)
pred_2 = model_2.predict(x_test)
prob_2 = best_model_2.predict_proba(x_test)[:, 1]

accuracy_2 = accuracy_score(y_test, pred_2)
auc_roc_2 = roc_auc_score(y_test, prob_2)

print(f'Valor de accuracy para DecisionTreeClassifier {accuracy_2}')
print(f'Valor de AUC_ROC para DecisionTreeClassifier {auc_roc_2}')

### GaussianNB

In [ ]:
def objective_3(trial):
    # Definir los hiperparámetros a optimizar
    var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-2)

    # Crear y entrenar el clasificador con los parámetros sugeridos
    model = GaussianNB(var_smoothing=var_smoothing)

    model.fit(x_train, y_train)

    # Predicciones y evaluación
    y_pred = model.predict(x_val)
    y_prob = model.predict_proba(x_val)[:, 1]  # Probabilidades para AUC-ROC

    # Calcular las métricas
    auc_roc = roc_auc_score(y_val, y_prob)
    accuracy = accuracy_score(y_val, y_pred)

    # Optuna optimiza para la métrica que se devuelve, en este caso, AUC-ROC
    return auc_roc

# Crear el estudio de Optuna
study_3 = optuna.create_study(direction='maximize')

# Ejecutar la optimización
study_3.optimize(objective_3, n_trials=50)

# Imprimir el mejor conjunto de hiperparámetros y las métricas correspondientes
print("Best hyperparameters:", study_3.best_params)
print("Best AUC-ROC:", study_3.best_value)

# Evaluar el modelo con los mejores hiperparámetros
best_params_3 = study_3.best_params
best_model_3 = GaussianNB(**best_params_3)
best_model_3.fit(x_train, y_train)

# Predicciones y métricas finales
y_pred_3 = best_model_3.predict(x_val)
y_prob_3 = best_model_3.predict_proba(x_val)[:, 1]

final_accuracy_3 = accuracy_score(y_val, y_pred_3)
final_auc_roc_3 = roc_auc_score(y_val, y_prob_3)

print("Final Accuracy:", final_accuracy_3)
print("Final AUC-ROC:", final_auc_roc_3)


#### Prueba con conjunto test

In [ ]:
model_3 = GaussianNB(var_smoothing = 4.2104179537407e-05)
model_3.fit(x_train, y_train)
pred_3 = model_3.predict(x_test)
prob_3 = best_model_3.predict_proba(x_test)[:, 1]

accuracy_3 = accuracy_score(y_test, pred_3)
auc_roc_3 = roc_auc_score(y_test, prob_3)

print(f'Valor de accuracy para DecisionTreeClassifier {accuracy_3}')
print(f'Valor de AUC_ROC para DecisionTreeClassifier {auc_roc_3}')

### Analisis Parte 7

- Se opto por usar inicialmente el modelo LasyClssifier para identificar los modelos que me sugerian este meotodo. De ellos solo elegí una debido a que no estoy familiarizado con la gran mayoría. 
- Se eleigieron los dos modelos que mejor rendimiento han presentado a lo largo de mis proyectos en tareas de clasificación, los cuales fueron RandomForestClassifer y DecisionTreeClassifier. 
- Se realizó una selección de las variables más relevantes para que se entrenaran con los conjuntos de data para RandomForestClassier y DecisionTreeClassifier.
- Se uso el modelo Optuna para encontrar los mejores hiperparametros de manera automatica. 
- Las metricas de evaluación fueron las señaladas en las indicaciones, la principal AUC-ROC y accuracy. 
- Despues de entrenarse los modelos con los conjuntos de entrenamiento, se evaluaron con los de validación y por ultimo con los test con los mejores hiperpametros configurados.
- El modelo con mejor rendimiento fue el de RandomDecisionTree, con un AUC-ROC de .866 con el conjunto de validación y con el de test fue de .864

<div class="alert alert-block alert-danger">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Vas por buen camino con los entrenamientos, pero estás usando la métrica de `accuracy` y no revisaste si los datos estaban balanceados, el accuracy no es una métrica recomendable si tienes un dataset desbalanceado en cuanto a las clases, te recomendaría utilizar el f1-score y poner cómo umbral un 0.75 de f1-score para tu mejor modelo.
 
Además, no se está revisando si el modelo no se está sobre ajustando, ¿Cómo haces esto? Debes realizar predicciones sobre el conjunto de datos de entrenamiento y obtener la métrica de evaluación (Esto a parte de las métricas con los datos de prueba y validación) con el fin de identificar si el modelo se ajusta correctamente ya que por ejemplo si tienes un modelo que con los datos de entrenamiento tiene una métrica perfecto pero con evaluación y pruebas tiene una métrica muy baja quiere decir que el modelo se está sobre-ajustando (overfitting) y debes aplicar alguna técnica para evitarlo.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor (2da Iteracion)</b> <a class=“tocSkip”></a>

Por favor, realiza las correcciones mencionadas en esta parte, debes evaluar para entrenamiento y prueba para ver si se está sobreajustando y debes evaluar con f1-score los modelos. Saludos!
</div>

## Paso 8. Conclusión.


1. La principal conclusión fue que esta muy definido el comportamiento de los usuarios para darse de baja. Se expresa muy bien esto en las graficas de distribución de densidad de las variables temporales derivadas de 'EndDate'. Sin embargo, debido a la metodologia para entrenar los modelos, no puede usarse estas valiosa información, ya que se incurriria en data leakage.
    * Solicitud: Si hay manera de poder procesar esta info para entrenar al modelo sin incurrir a data leakage, por favor compartemela.
    
2. Es muy relevante identificar cuales son las variables que más determinaron a churn, las cuales ya se mencionaron en el analisis del Paso 4. En este espacio explicare más a detalle las interpretaciones de las variables: 
    * Variables relación positiva ['year BeginDate', 'PaymenthMethod_Electronic check', 'InternetService_Fiber optic']:
        * 'year BeginDate': Indica que entre más reciente sea el cliente, mayor es la probabilidad de que se de de baja.
        * 'PaymenthMethod_Electronic check' y 'InternetService_Fiber optic': Si un cliente tiene estos servicios, implica que es más probable que se de de baja. INTERPRETACIÓN: En general podemos inferir que hay un problema con estos servicios, las causas pueden ser variadas, como un problema que el servicio de internet por fibra optica esta fallando ó lento. Lo mismo con el pago por cheque, puede que este ocurriendo un problema con este metodo de pago donde las personas hayan encontrados problemas al realizarlo y esto haya implicado una molestia para ellos.
    * Variables relación negativa ['TotalChargues', 'Type_two year', 'OnlineSecurity_Yes', 'TechSupport_Yes', 'OnlineBackup_Yes']:
        * 'TotalChargues': A mayor pago total, es menos probable que un cliente se de de baja. INTERPRETACION: Al parecer, los clientes que tienen cuentas más grandes, que implican más servicios, son los menos propensos a cancelar, lo que al mismo tiempo nos indica que a menor sea la cuenta de un cliente (y menores servicios contrate) mayor sea la probabilidad de que se de de baja.
        * 'OnlineSecurity_Yes' y 'OnlineBackup_Yes': Los clientes que tienen contratado estos servicios son los menos propensos a cancelar. INTERPRETACIÓN: Estos pueden ser los mejores productos con los que cuenta la empresa, por lo que pueden ser lo más adecuados a promover dado su alta tasa de retención.
        * 'TechSupport_Yes': Los usuarios que llamaron al servicio al cliente son los que menos cancelan. INTERPRETACIÓN: Parece ser que los usuarios que consultan sus problemas con el servicio al cliente quedan satisfechos, lo que habla de un buen rendimiento de esta de la empresa.
        
3. SUGERENCIAS:
    * Los clientes nuevos son propesos a cancelar, deben de generar estrategias para retener especificamente a los nuevos usuarios que tienen cuentas bajas. Otro enfoque puede ser el de modificar el modelo de negocio para que la empresa se enfoque especificamente a clientes que consumen varios servicios (cuentas altas), lo que se traduciria en un mejor uso de los recursos ya que estos son los clientes que mejor rendientos le da a la empresa por diferencia de costo de adquisicion de usuario y la esperanza de vida de este. 
    * Deben poner atención a los servicios con relación positiva, ya que estos son factores determinantes para que los clientes se den de baja. Pueden obtar por mejorarlos o eliminarlos. 
    * Pueden obtar por promover más los prodcutos que expresan mejor satisfacción para los clientes ó enfocar a la empresa a solo proveer estos servicios.
    * Dar a conocer o recordar frecuentemente a los usuarios que tienen disponible el servicio al cliente para cualquier problema que tengan, ya que es posible que quienes no llamen al servicio al cliente sea por ignorancia y si es que se hubieran marcado antes, no habrían cancelado. 
    * Es altamente recomendable que se realicen pruebas de muestreo de usuarios para probar estas y otras hipotesis que puedan surgir derivado de este analisis. 

## Plan de trabajo

- Paso 1. Inicialización: Cargar las librerias que serán usadas en el proyecto. +
- Paso 2. EDA: Analisis Explotario de Datos. En este paso se realizan acciones de limpieza, así como de analisis estadisticos con el fin de entender la data. +
- Paso 3. Normalización y codificación: Se estandarizan los datos a un mismo rango y se vuelvan comparables / equivalentes. Se codifican las variabes categoricas a numericas para habilitar su procesamiento. +
- Paso 4. Continuación de EDA
- Paso 5. La segmentación de datos en 3 conjuntos: entrenamiento, validación y prueba.
- Paso 6. Análisis de balance de clases. Una o más estrategias. +
- Paso 7. Selecciona los modelos con los que vas a trabajar y vas comparar. +
- Paso 8. Análisis en cada sección de trabajo y una conclusión general al final del proyecto. 